## Instructions: ##
  
1) Place this file at the root of the folder you started jupyter notebook in.  
2) Create a "data/" folder containing all turnstile data you want to parse.  

_Example Structure:_  
~~~~
/MTA_data_parser.ipynb  
/data  
/data/turnstile_180922.txt
~~~~

In [ ]:
from multiprocessing.pool import ThreadPool
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import pandas as pd
import pickle
import datetime
import requests
%matplotlib inline
sns.set()

# Put filepath here
data_path = 'data/'
fn = 'turnstile_180922.txt'

In [ ]:
# Download all log files given range

def gen_dates(start, end, days_between):
    '''Use datetime module to generate a list of dates yymmdd format given a start range,
    end range, and the num of days in between each date (start range is the base)
    :start: array of [int(year), int(mon), int(day)] specifying base start date
    :end: array of [int(year), int(mon), int(day)] specifying end date of range
    :days_between: how many days between each date (starting with start date), default = 7'''
    
    start = datetime.datetime(*start)
    end = datetime.datetime(*end)
    step = datetime.timedelta(days=days_between)
    dates = []
    
    while start < end:
        dates.append(start.strftime('%y%m%d'))
        start += step
    return dates

def download_logs(date):
    '''Given date input, download log file from MTA url:
    :date: a date str yymmdd'''
    
    print('Downloading', 'turnstile_' + date + '.txt')
    url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{}.txt'.format(date)
    response = requests.get(url)
    open('data/turnstile_' + date + '.txt', 'wb').write(response.content)

def parallel_dl(dates, threads = 8):
    '''Paralellize downloads to n threads
    :threads: number of concurrent download threads (default is 8)'''
    
    results = ThreadPool(threads).imap_unordered(download_logs, dates)

start = [2015,1,3]
end = [2018,9,23]
days_between = 7
dates = gen_dates(start, end, days_between)
# parallel_dl(dates)

In [ ]:
df = pd.read_csv(data_path + fn)
df = df.rename(columns=lambda x: x.strip())

In [ ]:
# Use to do preliminary inspection of data

def data_inspect(df, resampled=False):
    '''Preliminary inspection of data attributes.
    :resampled: If this is set to true, the data has been cleaned and some cols
    removed/added. Treat differently'''
    
    print("Summarize NAs:")
    print(df.isna().sum())
    
    u_station = df['STATION'].unique()
    print("\nNum Unique Stations:", len(u_station))
    print(u_station)
    
    u_desc = df['DESC'].unique()
    print("\n Unique Descriptions")
    print(u_desc)
    
    if resampled:
        print("\nUnique Time Vals per station")
        for s in u_station:
            u_time = df[df['STATION'] == s]['DATETIME'].unique()
            print('Station:', s)
            print(u_time, '\n')

    else:
        print("\nUnique Time Vals per station")
        for s in u_station:
            u_time = df[df['STATION'] == s]['TIME'].unique()
            print('Station:', s)
            print(u_time, '\n')

# data_inspect(df)

In [ ]:
# Organize data before using diff() to find absolute diff in entries/exits.
# Zero out counter rollovers (discard)
# Aggregate times to ensure 4 hour interval in-between

def clean_data(df, name='stop_data', threshold = 20000):
    '''Clean up dataframe by grouping by correct cols, finding change in counts
    from entries/exits, zeroing out implausible counts. Add a DATETIME column with
    a datetime object for ease of grouping in analysis, and to smooth irregular
    data reporting intervals. All of this is stored and pickled in a dict whose key
    is the unique station name, and value is a df with cols C/A, STATION, UNIT,
    SCP, DATE, TIME.
    :df: input raw df data to be cleaned
    :name: string for name of pickle dictionary that will be outputted
    :threshold: a threshold to throw away any |value| greater than '''
    
    stop_data = {}
    u_station = df['STATION'].unique()
    
    group_order = ['C/A', 'STATION', 'UNIT', 'SCP', 'DATE', 'TIME']
    # Sort by Control Area (station), Unit (Remote unit), SCP (turnstile), Date, Time
    print("Sorting cols")
    df = df.sort_values(by=group_order)
    
    # Substract next row by prev row for magnitude of change
    print("Finding row difference in ENTRIES col")
    df['DIFF_ENTRY'] = df['ENTRIES'].diff()
    
    # Add datetime col to resample data
    print("Converting to datetime object")
    df['DATETIME'] = df['DATE'] + ' ' + df['TIME']
    df['DATETIME'] = pd.to_datetime(df['DATETIME'])
    
    # Zero out turnstile errors/rollovers
    print("Cleaning outliers")
    mask1 = ((df['DIFF_ENTRY'] < 0) & (df['DIFF_ENTRY'] > -1*threshold))
    mask2 = ((df['DIFF_ENTRY'] < 0) | (df['DIFF_ENTRY'] > threshold))
    df.loc[mask1, 'DIFF_ENTRY'] *= -1
    df.loc[mask2, 'DIFF_ENTRY'] = 0
        
    for s in u_station:
        stop_data[s] = df[df['STATION'] == s][['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DATETIME']]
    
    with open(data_path + name, 'wb') as f:
        print('Pickling to', data_path + name, '...')
        pickle.dump(stop_data, f)
        print('Done!')

# clean_data(df, name='stop_data')

In [ ]:
def plot_stop_data(fn, stops, date_range=None, rule="4H", save=False):
    ''' Load and plot entry counts in pickled file containing dictionary with
    stop names as keys, and dataframe with columns C/A, UNIT, SCP, DIFF_ENTRY, DATETIME.
    :fn: this is the filename within the specified data dir above
    :stops: stops is an array containing string names of stops of concern
    :date_range: is a list with 2 entries, start date and end date,
    ["yyyy-mm-dd", "yyyy-mm-dd"]. Will be used to limit data range.
    :rule: this is the data aggregating option, default by to sum counts by 1 Day
    :save: give a string name to your graph to be saved as an svg; default does not save'''
    
    data = pickle.load(open(data_path+fn, 'rb'))
    
    # Pre-plot formattings
    plot_dims = (17,10)
    fig, ax = plt.subplots(figsize=plot_dims)
    y_name = 'Turnstile Entries'
    
    if 'H' in rule:
        x_format = '%H:%M'
        x_name = 'Hour of Day'
    else:
        x_format = '%m/%d/%y'
        x_name = 'Date'
    
    # Go through each stop of interest to plot
    for stop in stops:
        stop_data = data[stop]
        stop_data = stop_data[['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DATETIME']]
        
        # Slice data to range of concern
        if date_range:
            start_temp = date_range[0].split('-')
            start_temp = [int(x) for x in start_temp]
            end_temp = date_range[1].split('-')
            end_temp = [int(x) for x in end_temp]
            start = datetime.datetime(*start_temp)
            end = datetime.datetime(*end_temp)
            mask = (stop_data['DATETIME'] >= start) & (stop_data['DATETIME'] < end)
            stop_data = stop_data.loc[mask]
        
        # Resample to ensure regular time intervals (sum over irregular time intervals)
        grouped = stop_data.resample(rule, on='DATETIME', base=0).sum()

        # Plot
        date = grouped.index
        amount_entry = grouped['DIFF_ENTRY']
        sns.lineplot(x=date, y=amount_entry, label = stop, ax=ax)
    
    # Set formatting
    ax.set_xticklabels(labels=date, rotation=70)
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter(x_format))
    ax.set_xticks(date)
    ax.set_xlabel(x_name)
    ax.set_ylabel(y_name)
    
    # Save options
    if save:
        print('Saving graph to:', data_path + save + '.svg')
        plt.savefig(data_path + save + '.svg', format='svg')
    
stops = ['TIMES SQ-42 ST','34 ST-HERALD SQ',  '14 ST', '42 ST-BRYANT PK' , '14 ST-UNION SQ']
date_range = ['2018-1-08', '2018-9-3']

plot_stop_data('final_clean_data', stops, date_range, rule='1W', save='entry_count')

In [ ]:
def cumulative_plt(fn, date_range=None, max_stations=None, rule="4H", save=False):
    ''' Load and plot cumulative entry counts in pickled file containing dictionary with
    stop names as keys, and dataframe with columns C/A, UNIT, SCP, DIFF_ENTRY, DATETIME.
    :fn: this is the filename within the specified data dir above
    :date_range: is a list with 2 entries, start date and end date,
    ["yyyy-mm-dd", "yyyy-mm-dd"]. Will be used to limit data range.
    :rule: this is the data aggregating option, default by to sum counts by 1 Day
    :save: give a string name to your graph to be saved as an svg; default does not save'''
    
    data = pickle.load(open(data_path+fn, 'rb'))
    cumulative = []
    sum_total = 0
    if not max_stations:
        max_stations=len(data)

    # Pre-plot formattings
    plot_dims = (17,10)
    fig, ax = plt.subplots(figsize=plot_dims)
    y_name = 'Cumulative Entries'
    
    if 'H' in rule:
        x_format = '%H:%M'
        x_name = 'Hour of Day'
    else:
        x_format = '%m/%d/%y'
        x_name = 'Date'
    
    # Go through each stop of interest to plot
    for stop in data:
        stop_data = data[stop]
        stop_data = stop_data[['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DATETIME']]
        
        # Slice data to range of concern
        if date_range:
            start_temp = date_range[0].split('-')
            start_temp = [int(x) for x in start_temp]
            end_temp = date_range[1].split('-')
            end_temp = [int(x) for x in end_temp]
            start = datetime.datetime(*start_temp)
            end = datetime.datetime(*end_temp)
            mask = (stop_data['DATETIME'] >= start) & (stop_data['DATETIME'] < end)
            stop_data = stop_data.loc[mask]
        
        # Resample to ensure regular time intervals (sum over irregular time intervals)
        grouped = stop_data.resample(rule, on='DATETIME', base=0).sum()
        
        # Calculate cumulative over time period, then add to dict
        total_entries = grouped['DIFF_ENTRY'].sum()
        cumulative.append((stop, total_entries))
    
    # Turn cumulative list of tuples into sorted, descending df
    freq = pd.DataFrame(cumulative, columns=['STATION', 'CUMULATIVE ENTRIES'])
    freq.sort_values(by = ['CUMULATIVE ENTRIES'], ascending=False, inplace=True)
    freq.reset_index(inplace = True, drop=True)  
    freq['CUMSUM'] = freq['CUMULATIVE ENTRIES'].cumsum()
    freq['CUMPERC'] = freq['CUMSUM']/freq['CUMULATIVE ENTRIES'].sum()
    
    x_axis = freq['STATION'][:max_stations]
    height = freq['CUMPERC'][:max_stations]
    
    # Plot
    pal = sns.color_palette("Blues_d", n_colors=max_stations)
    plt.bar(x_axis, height, color=pal)
    plt.axvline(x=4.5, color = 'red', linestyle='dashed')
    ax.set_xticklabels(labels=x_axis, rotation=70)

    # Save options
    if save:
        print('Saving graph to:', data_path + save + '.svg')
        plt.savefig(data_path + save + '.svg', format='svg')

date_range = ['2018-1-08', '2018-9-3']
cumulative_plt('final_clean_data', date_range, max_stations=20, rule='1W', save='cumulative')

In [ ]:
def entries_for_census(df, census, date_range=None, rule='1D', save=False):
    mta_data = pickle.load(open(data_path+df, 'rb'))
    census_data = pd.read_csv(data_path+census, sep='\t')
    stops = list(census_data['station'])
    cum_entries = []
    above200 = []
    below200 = []
    
    for stop in stops:
        stop_data = mta_data[stop]
        stop_data = stop_data[['C/A', 'UNIT', 'SCP', 'DIFF_ENTRY', 'DATETIME']]
        
        # Slice data to range of concern
        if date_range:
            start_temp = date_range[0].split('-')
            start_temp = [int(x) for x in start_temp]
            end_temp = date_range[1].split('-')
            end_temp = [int(x) for x in end_temp]
            start = datetime.datetime(*start_temp)
            end = datetime.datetime(*end_temp)
            mask = (stop_data['DATETIME'] >= start) & (stop_data['DATETIME'] < end)
            stop_data = stop_data.loc[mask]
        
        # Resample to ensure regular time intervals (sum over irregular time intervals)
        grouped = stop_data.resample(rule, on='DATETIME', base=0).sum()
        
        # Calculate cumulative over time period, then add to dict
        total_entries = grouped['DIFF_ENTRY'].sum()
        above = int(census_data[census_data['station'] == stop]['Abv200K'])
        below = int(census_data[census_data['station'] == stop]['100K_200K'])
        cum_entries.append(total_entries)
        above200.append(above)
        below200.append(below)

    census_df = pd.DataFrame({'STATION':stops,
                              'CUMULATIVE ENTRY':cum_entries,
                              'MID INCOME': [a+b for a,b in list(zip(above200, below200))],
                              'TOTAL HOUSE': census_data['Total_households']})
    
    census_df['MID RATIO'] = census_df['MID INCOME']/census_df['TOTAL HOUSE']
    census_df.dropna(inplace=True)
    census_df.sort_values(by = ['CUMULATIVE ENTRY'], ascending=False, inplace=True)
    census_df.reset_index(inplace = True, drop=True)
    
    # Plot
    plot_dims = (17,10)
    fig, ax = plt.subplots(figsize=plot_dims)
    sns.scatterplot(x = 'CUMULATIVE ENTRY',
                    y = 'MID RATIO', 
                    data = census_df,
                    ax = ax)
    
    top_stations = ['59 ST', '86 ST', '59 ST COLUMBUS', 'CANAL ST',\
                    '23 ST', 'GRD CNTRL-42 ST', '34 ST-PENN STA', 'CITY HALL']
    
    # Create labels for top clustered stations
    for s in top_stations:
        mask = census_df['STATION'] == s
        c_entry = float(census_df[mask]['CUMULATIVE ENTRY'])
        m_ratio = float(census_df[mask]['MID RATIO'])
        ax.text(x=c_entry + 250000, y=m_ratio + .001, s=s)
        plt.scatter(c_entry, m_ratio, color='r')
        
    # Save options
    if save:
        print('Saving graph to:', data_path + save + '.svg')
        plt.savefig(data_path + save + '.svg', format='svg')
    
    return None

date_range = ['2018-1-08', '2018-9-3']
census_fp = "mta_census1.csv"
mta = "final_clean_data"

entries_for_census(mta, census_fp, date_range, save='census_and_income')